In [1]:
import os
import sys
sys.path.append('/root/capsule/code/beh_ephys_analysis')
from utils.beh_functions import parseSessionID, session_dirs, plot_session_in_time_all, plot_session_glm, transfer_nwb, get_unit_tbl
from utils.ephys_functions import make_summary_unit_tbl
from utils.opto_utils import opto_metrics, load_opto_sig
from unit_beh_analysis import plot_unit_beh_session, plot_alignments, burst_analysis
from behavior_and_time_alignment import beh_and_time_alignment_hopkins
import shutil
from aind_dynamic_foraging_data_utils.nwb_utils import load_nwb_from_filename
import pandas as pd
from joblib import Parallel, delayed
from aind_dynamic_foraging_basic_analysis.plot.plot_foraging_session import plot_foraging_session, plot_foraging_session_nwb
from aind_dynamic_foraging_basic_analysis.licks.lick_analysis import plot_lick_analysis
import datetime
import numpy as np
import matplotlib.pyplot as plt
import json
import pickle

In [2]:
session_id = 'behavior_ZS059_2021-04-12_14-57-43'
data_type = 'curated'
session_id_example   = 'behavior_754897_2025-03-13_11-20-42'
session_dir = session_dirs(session_id)
session_dir['ephys_mat']

Old data, using hopkins formats


'/root/capsule/data/behavior_ZS059_2021-04-12_14-57-43_raw_data/ecephys/sorted/session/mZS059d20210412_sessionData_nL.mat'

In [3]:
beh_and_time_alignment_hopkins(session_id)

Old data, using hopkins formats
behavior_ZS059_2021-04-12_14-57-43
(32, 79223680)


/opt/conda/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/numpy/lib/_histograms_impl.py:902: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


In [5]:
opto_metrics_session = opto_metrics(session_id_example, 'curated')
opto_sigs_session = load_opto_sig(session_id_example, 'curated')

Single experiment found: experiment1, recording1
Single experiment found: experiment1, recording1


In [6]:
session_id = 'behavior_ZS061_2021-04-14_16-04-22'
data_type = 'curated'
def convert_hopkins_opto(session_id, data_type='curated'): # make fake opto_metrcs for hopkins data
    session_dir = session_dirs(session_id)
    unit_nwb = load_nwb_from_filename(session_dir['nwb_dir_curated'])
    unit_tbl = unit_nwb.units[:]
    # opto_metrics
    cols = ['unit_id', 'p_resp', 'lat_resp', 'euc_max_p', 'corr_max_p', 'pulse_width', 'bl_max_p', 'p_mean']
    all_neurons = []
    # ['bl_max_p'] 
    for unit_ind, row in unit_tbl.iterrows():
        curr_dict = row[cols].to_dict()
        curr_dict = {
            key: (value[0] if isinstance(value, np.ndarray) else value)
            for key, value in curr_dict.items()
        }
        curr_df = pd.DataFrame(curr_dict)
        curr_df['resp_p_bl'] = curr_df['p_resp'] - curr_df['bl_max_p']
        curr_df.drop(['bl_max_p'], axis=1, inplace=True)
        all_neurons.append(curr_df)
    opto_metrics_tbl = pd.concat(all_neurons, axis=0, ignore_index=True)
    opto_file = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session_id}_opto_tagging_metrics.pkl')
    # with open(unit_tbl_dir, 'rb') as f:
    #     unit_data = pickle.load(f)
    # qm_data = unit_data['opto_tagging_df_metrics']
    opto_metrics_tbl.rename(columns={'p_resp': 'resp_p', 'lat_resp': 'resp_lat', 'euc_max_p': 'euclidean_norm', 'corr_max_p': 'correlation', 'p_mean': 'mean_p'}, inplace=True)
    opto_tagging_data = {'opto_tagging_df': None, 'opto_tagging_df_metrics': opto_metrics_tbl}
    with open(opto_file, 'wb') as f:
        pickle.dump(opto_tagging_data, f)
    # opto sigs
    opto_sigs_file = os.path.join(session_dir[f'opto_dir_{data_type}'], f'{session_id}_opto_sigs.pkl')
    neuron_sig_df = pd.DataFrame({'unit_id': unit_tbl['unit_id'].to_list(), 'p_sig_count': [5]*len(unit_tbl)})
    with open(opto_sigs_file, 'wb') as f:
        pickle.dump(neuron_sig_df, f)

convert_hopkins_opto(session_id, data_type)

Old data, using hopkins formats


In [8]:
# test
opto_metrics_tbl = opto_metrics(session_id, data_type).qm_data
opto_metrics_tbl
opto_sigs_session = load_opto_sig(session_id, data_type)
opto_sigs_session.opto_sigs

,unit_id,p_sig_count
0,TT4_SS_01,5


In [ ]:
# get summary unit table's columns # just needs to be run once
example_tbl = get_unit_tbl('behavior_761038_2025-04-15_10-25-11', 'curated')
col_list = example_tbl.columns.tolist()
with open("summary_col_list.json", "w") as f:
    json.dump(col_list, f)

Selected experiment1 recording1, length:3362.95


In [7]:
# critical functions
# check time alignment
# beh_and_time_alignment_hopkins(session_id)
# # # make summary unit table
# make_summary_unit_tbl(session_id)
# make opto tagging data
convert_hopkins_opto(session_id, data_type)

In [9]:
# test:
plot_unit_beh_session('behavior_ZS061_2021-04-20_15-49-57', align_name='go_cue')

KeyboardInterrupt: 

In [3]:
# critical batch processing
# get session list
session_df = pd.read_csv('/root/capsule/code/data_management/hopkins_session_assets.csv')
session_ids = session_df['session_id'].to_list()
# functions to use
def process(session_id):
    session_dir = session_dirs(session_id);
    if os.path.exists(session_dir['nwb_dir_raw']) and get_unit_tbl(session_id, 'curated') is not None:
        # try:
        # transfer_nwb(session_id);
        beh_and_time_alignment_hopkins(session_id, ephys_cut=[0, 0]);
        # make_summary_unit_tbl(session)
        # plot_unit_beh_session(session_id, align_name='go_cue')
        # plot_unit_beh_session(session_id, align_name='response')
        # plot_alignments(session_id)
        # convert_hopkins_opto(session_id, 'curated')
        # burst_analysis(session_id, 'curated')
        print(f'Processed {session_id}')
        plt.close('all')
        # except:
        #     print(f'{session_id} failed')

In [4]:
process('behavior_ZS059_2021-03-29_14-41-57')

Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021-03-29_14-41-57


Processed behavior_ZS059_2021-03-29_14-41-57
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
No unit table found for behavior_ZS059_2021-03-26_13-52-11 in curated data.
Old data, using hopkins formats
No unit table found for behavior_ZS059_2021-03-25_14-40-45 in curated data.
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021-03-30_14-57-13
Old data, using hopkins formats
No unit table found for behavior_ZS059_2021-04-02_15-05-16 in curated data.
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021-03-28_14-18-58
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
behavior_ZS059_2021-03-31_14-32-51
Old data, using hopkins formats
Old data, using hopkins formats
Old data, using hopkins formats
No unit table 

In [5]:
# execution
# for session_ind, session_id  in enumerate(session_ids):
#     session_dir = session_dirs(session_id);
#     if os.path.exists(session_dir['nwb_dir_raw']):
#         process(session_id)
Parallel(n_jobs=10)(delayed(process)(session_id) for session_id in session_ids)

/opt/conda/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/conda/lib/python3.10/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/opt/conda/lib/python3.10/site-packages/numpy/lib/_histograms_impl.py:902: RuntimeWarning: invalid value encountered in divide
  return n/db/n.sum(), bin_edges


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,